In [1]:
import glob
import os
import numpy as np
from PIL import Image
import skimage
import skimage.io
import scipy.misc
import imageio
import warnings; warnings.simplefilter('ignore')

list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

list_dict = {}
for i in range(10):
    key = list_classes[i]
    v = [0,0,0,0,0,0,0,0,0,0]
    v[i] = 1
    list_dict[key] = v
print(list_dict)

train_dir = '../input/crop_train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
train_data_cnt = len(train_files)
from sklearn.utils import shuffle
train_files = shuffle(train_files,random_state=66)
train_cnt = int(train_data_cnt * 0.8)
valid_cnt = train_data_cnt - train_cnt
BATCH_SIZE = 32
CROP_LEN = 224

def random_crop(im_array):
    # crop
    x_range = im_array.shape[0] - CROP_LEN
    y_range = im_array.shape[1] - CROP_LEN
    # print(x_range,y_range)
    a = np.random.randint(x_range)
    b = a + CROP_LEN
    c = np.random.randint(y_range)
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array

def center_crop(im_array):
    center_x = im_array.shape[0] // 2
    center_y = im_array.shape[1] // 2
    half_crop = CROP_LEN // 2
    a = center_x - half_crop
    b = a + CROP_LEN
    c = center_y - half_crop
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array
    

def get_img(img_path, train_flag = True):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    
    # train or valid
    if train_flag is True:
        final_img = random_crop(im_array)
    else:
        final_img = center_crop(im_array)
    
    final_img = final_img/255.0
    return final_img

# 读取测试数据中间 224
def get_test_img(img_path):
    # read img
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    final_img = center_crop(im_array)
    final_img = final_img/255.0
    return final_img

def get_y(img_path):
    n = img_path.split('/')[-2]
    return list_dict[n]

def data_gen(file_list, batch_size=BATCH_SIZE, train_flag = True):
    curr_idx = 0
    data_cnt = len(file_list)
    while True:
        if curr_idx + batch_size > data_cnt:
            start_idx = data_cnt-batch_size
            end_idx = data_cnt
            curr_idx = 0
        else:
            start_idx = curr_idx
            end_idx = curr_idx + batch_size
            curr_idx += batch_size
        curr_fl = file_list[start_idx:end_idx]
        curr_x = [get_img(p,train_flag) for p in curr_fl]
        curr_x = np.array(curr_x,dtype='float32')
        curr_y = np.array([get_y(p) for p in curr_fl])
        yield curr_x,curr_y

train_gen = data_gen(train_files[:train_cnt], BATCH_SIZE, True)
valid_gen = data_gen(train_files[train_cnt:], BATCH_SIZE, False)
import math
train_step = math.ceil(train_cnt*0.5/BATCH_SIZE)
valid_step = math.ceil(valid_cnt/BATCH_SIZE)
    

{'HTC-1-M7': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'LG-Nexus-5x': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 'iPhone-6': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 'Motorola-Droid-Maxx': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 'Samsung-Galaxy-S4': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'Motorola-X': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'Samsung-Galaxy-Note3': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'iPhone-4s': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'Motorola-Nexus-6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 'Sony-NEX-7': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [2]:
# def model
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [3]:
model_p = 'best_cnn2_crop.h5'  # copy best_cnn2_manip

if os.path.exists(model_p):
    model = load_model(model_p)
    #K.set_value(model.optimizer.lr, 0.0001)
    print('load',model_p)

load best_cnn2_crop.h5


In [4]:
model_chk = ModelCheckpoint(filepath=model_p, monitor='val_acc', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, verbose=1,
                              patience=3, min_lr=0.00000001)

model.fit_generator(train_gen,
          steps_per_epoch = train_step,
          epochs=100,
          validation_data = valid_gen,
          validation_steps = valid_step,
          callbacks=[model_chk,reduce_lr]
         )
           

Epoch 1/100
550/550 [==============================] - 152s 276ms/step - loss: 0.8406 - acc: 0.7247 - val_loss: 0.7787 - val_acc: 0.7326
Epoch 2/100
550/550 [==============================] - 154s 279ms/step - loss: 0.8202 - acc: 0.7308 - val_loss: 0.7730 - val_acc: 0.7420
Epoch 3/100
550/550 [==============================] - 153s 279ms/step - loss: 0.8053 - acc: 0.7328 - val_loss: 0.7524 - val_acc: 0.7411
Epoch 4/100
550/550 [==============================] - 149s 271ms/step - loss: 0.8027 - acc: 0.7364 - val_loss: 0.7924 - val_acc: 0.7318
Epoch 5/100
550/550 [==============================] - 148s 270ms/step - loss: 0.7895 - acc: 0.7424 - val_loss: 0.7481 - val_acc: 0.7474
Epoch 6/100
550/550 [==============================] - 148s 270ms/step - loss: 0.7813 - acc: 0.7425 - val_loss: 0.7598 - val_acc: 0.7450
Epoch 7/100
550/550 [==============================] - 149s 271ms/step - loss: 0.7714 - acc: 0.7411 - val_loss: 0.7263 - val_acc: 0.7575
Epoch 8/100
550/550 [====================

Epoch 31/100
550/550 [==============================] - 146s 266ms/step - loss: 0.6387 - acc: 0.7887 - val_loss: 0.6449 - val_acc: 0.7823
Epoch 32/100
550/550 [==============================] - 146s 265ms/step - loss: 0.6402 - acc: 0.7877 - val_loss: 0.6520 - val_acc: 0.7803
Epoch 33/100
550/550 [==============================] - 146s 265ms/step - loss: 0.6322 - acc: 0.7918 - val_loss: 0.6499 - val_acc: 0.7792
Epoch 34/100
550/550 [==============================] - 146s 265ms/step - loss: 0.6341 - acc: 0.7901 - val_loss: 0.6389 - val_acc: 0.7858
Epoch 35/100
550/550 [==============================] - 146s 265ms/step - loss: 0.6155 - acc: 0.7989 - val_loss: 0.6251 - val_acc: 0.7894
Epoch 36/100
550/550 [==============================] - 146s 265ms/step - loss: 0.6225 - acc: 0.7909 - val_loss: 0.6087 - val_acc: 0.7969
Epoch 37/100
550/550 [==============================] - 146s 266ms/step - loss: 0.6129 - acc: 0.7974 - val_loss: 0.6029 - val_acc: 0.7975
Epoch 38/100
550/550 [============

Epoch 61/100
550/550 [==============================] - 147s 268ms/step - loss: 0.5144 - acc: 0.8303 - val_loss: 0.5404 - val_acc: 0.8199
Epoch 62/100
550/550 [==============================] - 148s 269ms/step - loss: 0.5169 - acc: 0.8310 - val_loss: 0.5299 - val_acc: 0.8224
Epoch 63/100
550/550 [==============================] - 147s 268ms/step - loss: 0.5116 - acc: 0.8345 - val_loss: 0.5488 - val_acc: 0.8203
Epoch 64/100
550/550 [==============================] - 148s 269ms/step - loss: 0.5125 - acc: 0.8302 - val_loss: 0.5326 - val_acc: 0.8243
Epoch 65/100
550/550 [==============================] - 149s 270ms/step - loss: 0.5198 - acc: 0.8274 - val_loss: 0.5313 - val_acc: 0.8232
Epoch 66/100
550/550 [==============================] - 147s 268ms/step - loss: 0.5107 - acc: 0.8301 - val_loss: 0.5233 - val_acc: 0.8269
Epoch 67/100
550/550 [==============================] - 149s 271ms/step - loss: 0.5001 - acc: 0.8360 - val_loss: 0.5576 - val_acc: 0.8155
Epoch 68/100
550/550 [============

550/550 [==============================] - 173s 315ms/step - loss: 0.4546 - acc: 0.8509 - val_loss: 0.5052 - val_acc: 0.8367
Epoch 92/100
550/550 [==============================] - 176s 319ms/step - loss: 0.4579 - acc: 0.8476 - val_loss: 0.4954 - val_acc: 0.8383
Epoch 93/100
550/550 [==============================] - 173s 315ms/step - loss: 0.4583 - acc: 0.8508 - val_loss: 0.5011 - val_acc: 0.8380
Epoch 94/100
550/550 [==============================] - 176s 320ms/step - loss: 0.4659 - acc: 0.8471 - val_loss: 0.4947 - val_acc: 0.8424
Epoch 95/100
549/550 [============================>.] - ETA: 0s - loss: 0.4618 - acc: 0.8517Epoch 00095: val_acc did not improve

Epoch 00095: reducing learning rate to 1.249999968422344e-05.
550/550 [==============================] - 175s 317ms/step - loss: 0.4619 - acc: 0.8518 - val_loss: 0.4938 - val_acc: 0.8411
Epoch 96/100
550/550 [==============================] - 179s 326ms/step - loss: 0.4532 - acc: 0.8499 - val_loss: 0.4899 - val_acc: 0.8410
Epoch 

In [6]:
import pandas as pd
best_model = load_model(model_p)
print('load done')
# TTA
def get_tta_img(img_path):
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    im_array = im_array/255.0
    img_list = []
    img_list.append(center_crop(im_array))
#     for i in range(4):
#         for j in range(3):
#             a = i*72
#             b = a + 224
#             c = j*96
#             d = c + 224
#             #print(a,b,c,d)
#             img_list.append(im_array[a:b,c:d,:])
    img_list.append(im_array[0:CROP_LEN,0:CROP_LEN,:])
    img_list.append(im_array[0:CROP_LEN,512-CROP_LEN:512,:])
    img_list.append(im_array[512-CROP_LEN:512,0:CROP_LEN,:])
    img_list.append(im_array[512-CROP_LEN:512,512-CROP_LEN:512,:])
    return np.array(img_list)
tta_img = get_tta_img(test_files[0])
print(tta_img.shape)
tta_res = best_model.predict(tta_img)
print(tta_res.shape)
print(np.sum(tta_res,axis=0))

test_y = []
for img_p in test_files:
    tmp_x = get_tta_img(img_p)
    tmp_y = best_model.predict(tmp_x)
    tmp_y = np.sum(tmp_y,axis=0)
    test_y.append(tmp_y)

test_y = np.array(test_y)
print(test_y[:5])

y_res = np.argmax(test_y,axis=1)
y_res = [list_classes[i] for i in y_res]
df = pd.read_csv('../input/sample_submission.csv')
f_name = [p.split('/')[-1] for p in test_files]
df['fname'] = f_name
df['camera'] = y_res
print(df.head())
df.to_csv('../results/cnn2_manip_crop_tta.csv',index=False)

load done
(5, 224, 224, 3)
(5, 10)
[ 0.14830309  0.20949683  0.23779932  0.06523371  0.23834926  0.00345666
  0.11749478  3.42020273  0.39015874  0.1695046 ]
[[  1.48303092e-01   2.09496826e-01   2.37799317e-01   6.52337074e-02
    2.38349259e-01   3.45666334e-03   1.17494784e-01   3.42020273e+00
    3.90158743e-01   1.69504598e-01]
 [  7.14331109e-05   4.08905149e-02   5.07117584e-02   6.16930483e-04
    1.83537558e-01   3.33294153e+00   3.44741255e-01   4.74592671e-02
    7.99714995e-04   9.98230100e-01]
 [  7.98577815e-02   5.05645806e-03   1.22439551e+00   3.65436985e-03
    3.86808068e-02   1.61044430e-02   2.76259518e+00   8.47929835e-01
    3.21376743e-03   1.85120888e-02]
 [  2.76058972e-06   5.62600754e-02   3.98852956e-03   5.87578192e-02
    1.56140857e-04   1.46147900e-03   2.69644265e-03   1.01082344e-02
    4.77706337e+00   8.95049274e-02]
 [  2.03594953e-11   4.73902893e+00   5.51029574e-04   1.69557765e-01
    6.71024172e-05   3.93004620e-06   8.87574672e-08   2.1560356